In [1]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

In [4]:
logdir = 'logs/hparam_tuning'
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [2]:
def train_test_model(hparams):
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(32, 3, activation='relu', input_shape=(20, 20, 1)))         # 卷积核的个数 => 输出的维度
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(3))
    model.compile(optimizer='adam', 
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])
            
    model.fit(x_train, y_train, epochs=epoch, validation_data=(x_test, y_test), initial_epoch=10, callbacks = [tf.keras.callbacks.TensorBoard(logdir), hp.KerasCallback(logdir, hparams)])
    _, accuracy = model.evaluate(x_test, y_test)
    return accuracy

In [ ]:
%tensorboard --logdir logs/hparam_tuning